In [1]:
!pip install -U langchain_community tiktoken langchain-openai langchain-cohere langchainhub chromadb langchain langgraph  tavily-python python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.5/602.5 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import pandas as pd
from docx import Document
from langchain.schema import Document
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import load_prompt
from langchain_core.output_parsers import StrOutputParser


**Create Index**

In [4]:
# Set embeddings
embd = OpenAIEmbeddings()

def load_file(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            return f.read()
    else:
        raise ValueError("Formato di file non supportato. Usa solo xlsx o DOCX.")

# Esempio di utilizzo
docs = [load_file("Commenti.txt"),load_file("Viaggio in barca a vela.txt")]

docs_list = [Document(page_content=doc) for doc in docs]
#docs_list = [doc for sublist in docs for doc in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embd,
)
retriever = vectorstore.as_retriever()


**LLMs**

In [5]:
# Data model
class RouteQuery(BaseModel):
    """Indirizza una query dell'utente alla fonte di dati più pertinente."""

    datasource: Literal["vectorstore"] = Field(
        ...,
        description="Given a user question choose to route it to a vectorstore.",
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """Tu sei esperto nell'indirizzare una domanda dell'utente a un vectorstore oppure dire che: 'non sei in grado di rispondere a questa domanda'.
Il vectorstore contiene documenti relativi ad Commenti di un video youtube, trascrizione del video di due ragazzi che stanno facendo e hanno fatto un viaggio in barca a vela.
Utilizzare il vectorstore per le domande su questi argomenti. Altrimenti, utilizzare dire che: 'non sei in grado di rispondere a questa domanda'."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "In quali posti hanno navigato?"}
    )
)
print(question_router.invoke({"question": "che ricetta di cucina hanno fatto?"}))

datasource='vectorstore'
datasource='vectorstore'


**Retrieval Grader**

In [6]:
# Data model
class GradeDocuments(BaseModel):
    """Punteggio binario per la verifica della rilevanza dei documenti recuperati."""

    binary_score: str = Field(
        description="I documenti sono rilevanti alla domanda, 'si' o 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """Siete un classificatore che valuta la pertinenza di un documento recuperato rispetto a una domanda dell'utente. \n
    Se il documento contiene parole chiave o significati semantici correlati alla domanda dell'utente, lo classifica come pertinente. \n
    Non è necessario che si tratti di un test rigoroso. L'obiettivo è filtrare i Retrievals errati. \n
    Dare un punteggio binario “sì” o “no” per indicare se il documento è rilevante per la domanda."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documenti: \n\n {document} \n\n Domanda user: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

<ipython-input-6-664cb9d4e6ab>:31: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


binary_score='no'


**Generate**

In [17]:
# Prompt
prompt = load_prompt("prompt.json")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Customer personas:

1. Who: Persona 1
   Goal: Fare un viaggio che desiderava fare
   Barrier: Il lavoro non permette di lavorare in barca

2. Who: Persona 2
   Goal: Progettare un viaggio del genere
   Barrier: Competenze nautiche limitate

3. Who: Persona 3
   Goal: Seguire viaggi non banali
   Barrier: Sentimento di tristezza e malinconia

Keywords:
- Navigare
- Solcato i mari
- Viaggio verso sud
- Viaggio in barca a vela
- Emozioni in barca a vela
- Residenza in barca
- Esperienza nautica
- Coraggio di viaggiare
- Posti caldi
- Atlantico
- Aeroporto
- Viaggiatori non numerosissimi
- Competenze nautiche
- Curiosità
- Beneteau
- Ciao bianco
- Tristezza e malinconia
